In [74]:
from pathlib import Path
# Añadir carpeta raíz del proyecto al path
import sys, pathlib
PROJECT_ROOT = pathlib.Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
from src import config as cfg

#import importlib
#importlib.reload(cfg)

ruta_tickers = cfg.DATA / "raw" / "tickers.txt"

if ruta_tickers.exists():
    with open(ruta_tickers, "r") as f:
        tickers = [line.strip() for line in f if line.strip()]
    print("🟢 Se cargaron", len(tickers), "tickers:")
    print(tickers)
else:
    print("❌ Archivo no encontrado:", ruta_tickers.resolve())



🟢 Se cargaron 40 tickers:
['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'META', 'AMZN', 'TSLA', 'JPM', 'JNJ', 'V', 'PG', 'PEP', 'KO', 'MA', 'UNH', 'HD', 'BAC', 'MRK', 'DIS', 'XOM', 'VZ', 'PFE', 'CSCO', 'WMT', 'ABT', 'NFLX', 'ADBE', 'CRM', 'T', 'INTC', 'COST', 'MCD', 'WFC', 'IBM', 'ORCL', 'NKE', 'LIN', 'CVX', 'BTC-USD', 'ETH-USD']


In [76]:
import yfinance as yf
import pandas as pd

# Descargar todos los tickers
df = yf.download(
    tickers,
    start="2010-01-01",
    auto_adjust=False,   # controlas tú la columna ajustada
    progress=False
)

# Extraer precios ajustados correctamente
if "Adj Close" in df.columns.get_level_values(0):
    df_close = df["Adj Close"]
else:
    df_close = df["Close"]

# Relleno específico para criptomonedas (BTC y ETH)
for ticker in ["BTC-USD", "ETH-USD"]:
    if ticker in df_close.columns:
        first_valid = df_close[ticker].first_valid_index()
        if first_valid:
            # Rellenar solo ANTES del primer valor real
            df_close.loc[:first_valid, ticker] = df_close.loc[:first_valid, ticker].fillna(1.0)
            print(f"ℹ️ Se rellenaron los NaN anteriores a {first_valid.date()} en {ticker} con valor 1.0")

print("✅ Datos descargados con forma:", df_close.shape)



ℹ️ Se rellenaron los NaN anteriores a 2014-09-17 en BTC-USD con valor 1.0
ℹ️ Se rellenaron los NaN anteriores a 2017-11-09 en ETH-USD con valor 1.0
✅ Datos descargados con forma: (5120, 40)


In [78]:
from src import config as cfg

# Asegura la carpeta raw (data/raw)
(cfg.DATA / "raw").mkdir(parents=True, exist_ok=True)

# Guardar precios de cierre en formato parquet
df_close.to_parquet(cfg.DATA / "raw" / "prices.parquet")

print("✅ Archivo guardado en:", cfg.DATA / "raw" / "prices.parquet")

✅ Archivo guardado en: C:\Users\ferra\Documents\TFM\data\raw\prices.parquet


In [87]:
# ----------------------------------------------------------
# Descargar VIX (volatilidad implícita del S&P500)
# ----------------------------------------------------------
vix = yf.download("^VIX", start="2010-01-01", auto_adjust=False, progress=False)["Adj Close"]
vix.name = "VIX"
vix = vix.ffill()
vix.to_parquet(cfg.DATA / "raw" / "vix.parquet")
print("✅ VIX guardado en:", cfg.DATA / "raw" / "vix.parquet")


✅ VIX guardado en: C:\Users\ferra\Documents\TFM\data\raw\vix.parquet


In [97]:
# ----------------------------------------------------------
# Tipos de interés a corto plazo (opcional)
# ----------------------------------------------------------
raw_irx = yf.download("^IRX", start="2010-01-01", progress=False)
irx = raw_irx["Adj Close"] if "Adj Close" in raw_irx.columns else raw_irx["Close"]
irx.name = "IRX"
irx = irx.ffill()
irx.to_parquet(cfg.DATA / "raw" / "fedfunds.parquet")
print("✅ FedFunds guardado en:", cfg.DATA / "raw" / "fedfunds.parquet")


C:\Users\ferra\AppData\Local\Temp\ipykernel_21476\2785725318.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_irx = yf.download("^IRX", start="2010-01-01", progress=False)


✅ FedFunds guardado en: C:\Users\ferra\Documents\TFM\data\raw\fedfunds.parquet
